![price-and-cut-algo](price-and-cut-algo.png)

In [1]:
import pulp

# --- 1. FAKE DATA (To make the example runnable) ---
# In a real problem, this 'known_schedules' list would be
# populated by the Subproblem.

# We represent each "column" (schedule j) as a Python dictionary
schedule_1 = {
    "id": "Sched_1_Mon",
    "B_j": 240,  # Total minutes (profit) of surgeries
    "day": "Mon",
    "surgeries": [101, 102],  # List of surgery IDs
    # Eq (5) data: {surgeon: total_minutes}
    "surgeon_work": {"Dr_A": 120, "Dr_B": 120},
    # Eq (6) data: {(surgeon, day, time_slot): 1 if busy}
    "surgeon_busy_times": {
        ("Dr_A", "Mon", 9): 1, ("Dr_A", "Mon", 10): 1,  # Dr_A busy 9-11
        ("Dr_B", "Mon", 11): 1, ("Dr_B", "Mon", 12): 1   # Dr_B busy 11-1
    }
}

schedule_2 = {
    "id": "Sched_2_Mon",
    "B_j": 300,
    "day": "Mon",
    "surgeries": [103],  # A mandatory surgery
    "surgeon_work": {"Dr_A": 300},
    "surgeon_busy_times": {
        ("Dr_A", "Mon", 11): 1, ("Dr_A", "Mon", 12): 1, ("Dr_A", "Mon", 13): 1,
        ("Dr_A", "Mon", 14): 1, ("Dr_A", "Mon", 15): 1   # Dr_A busy 11-4
    }
}

# The list of all columns the Master Problem currently knows about
known_schedules = [schedule_1, schedule_2]

# --- FAKE Problem Parameters ---
MANDATORY_SURGERIES = [103, 101, 102]             # Set I_1
OPTIONAL_SURGERIES = []         # Set I_2
ALL_SURGEONS = ["Dr_A", "Dr_B"]         # Set L
ALL_DAYS = ["Mon", "Tue"]               # Set D
ALL_TIMES = [9, 10, 11, 12, 13, 14, 15, 16] # Set T_d

# Resource Limits (RHS of constraints)
K_d = {"Mon": 5, "Tue": 5}  # Max 5 ORs per day (set of ORs for day d)
A_ld = {  # Max minutes for surgeon 'l' on day 'd'
    ("Dr_A", "Mon"): 480, ("Dr_B", "Mon"): 480,
    ("Dr_A", "Tue"): 480, ("Dr_B", "Tue"): 480,
}


def build_master_lp(known_schedules, mandatory_surgeries, optional_surgeries,
                    all_surgeons, all_days, all_times, K_d, A_ld):
    """
    Builds the Master Problem LP model from the paper.
    """

    # --- 1. Create the Problem ---
    prob = pulp.LpProblem("Master_Problem_LP", pulp.LpMaximize)

    # --- 2. Create Variables (x_j) ---
    # x_j = 1 if schedule j is used. Solved as LP (Continuous).
    # This is variable x_j from [cite: 166]
    sched_vars = pulp.LpVariable.dicts(
        "Schedule",
        [s["id"] for s in known_schedules],
        lowBound=0,
        cat='Continuous'
    )

    # --- 3. Objective Function (Eq. 1) ---
    # Maximize total scheduled surgery time: Max sum(B_j * x_j) 
    prob += pulp.lpSum(
        s["B_j"] * sched_vars[s["id"]] for s in known_schedules
    ), "Total_Surgery_Time"

    # --- 4. Constraints (This is where the dual prices come from) ---

    # === Constraint (2) & (3) -> dual price is pi_i ===
    # (2) Mandatory: sum(x_j for j containing i) = 1 [cite: 189, 190]
    for i in mandatory_surgeries:
        prob += pulp.lpSum(
            sched_vars[s["id"]] for s in known_schedules if i in s["surgeries"]
        ) == 1, f"Pi_i_Mandatory_{i}" # <-- Name generates the pi

    # (3) Optional: sum(x_j for j containing i) <= 1 [cite: 189, 190]
    for i in optional_surgeries:
        prob += pulp.lpSum(
            sched_vars[s["id"]] for s in known_schedules if i in s["surgeries"]
        ) <= 1, f"Pi_i_Optional_{i}" # <-- Name generates the pi

    # === Constraint (4) -> dual price is pi_d ===
    # Number of schedules on day d <= Number of ORs on day d [cite: 189, 192]
    for d in all_days:
        prob += pulp.lpSum(
            sched_vars[s["id"]] for s in known_schedules if s["day"] == d
        ) <= K_d[d], f"Pi_d_OR_Limit_{d}" # <-- Name generates the pi

    # === Constraint (5) -> dual price is pi_ld ===
    # Total surgeon work on day d <= Max hours for surgeon on day d [cite: 189, 193]
    for l in all_surgeons:
        for d in all_days:
            prob += pulp.lpSum(
                s["surgeon_work"].get(l, 0) * sched_vars[s["id"]]
                for s in known_schedules if s["day"] == d
            ) <= A_ld[(l, d)], f"Pi_ld_Surgeon_Hours_{l}_{d}" # <-- Name

    # === Constraint (6) -> dual price is pi_ldt ===
    # Prevents surgeon overlap [cite: 189, 193]
    # sum(x_j if surgeon l is busy at time t on day d in schedule j) <= 1
    for l in all_surgeons:
        for d in all_days:
            for t in all_times:
                prob += pulp.lpSum(
                    # s["surgeon_busy_times"].get((l, d, t), 0) checks if
                    # schedule 's' uses surgeon 'l' at day 'd', time 't'
                    s["surgeon_busy_times"].get((l, d, t), 0) * sched_vars[s["id"]]
                    for s in known_schedules if s["day"] == d
                ) <= 1, f"Pi_ldt_Surgeon_Overlap_{l}_{d}_{t}" # <-- Name

    return prob

if __name__ == "__main__":    
    # 1. Build the problem one last time, with ALL 500 schedules
    final_prob = build_master_lp(known_schedules, MANDATORY_SURGERIES, OPTIONAL_SURGERIES,
                                ALL_SURGEONS, ALL_DAYS, ALL_TIMES, K_d, A_ld)

    # 2. Change all variables to be Integer
    for v in final_prob.variables():
        v.cat = 'Integer'
        # You'd also set an upper bound
        v.upBound = 1 # We can only use a specific daily schedule once

    # 3. Solve this as an Integer Problem
    print("Solving the FINAL Integer Problem for the real answer...")
    final_prob.solve()

    # --- CHECK THE STATUS ---
    print(f"Status: {pulp.LpStatus[final_prob.status]}")

    # 4. THIS IS YOUR REAL ANSWER
    if final_prob.status == pulp.LpStatusOptimal:
        print("\n--- FINAL OR ROOM ALLOCATION ---")
        for v in final_prob.variables():
            if v.value() > 0.9: # i.e., if the variable = 1.0
                print(f"USE SCHEDULE: {v.name}")
    else:
        print("\nNo valid schedule allocation found.")
        print("The problem is likely infeasible.")
   

Solving the FINAL Integer Problem for the real answer...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/matthewkeller/4A/bme-411/411-final-project/or-env/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/_m/zyrbphls3w34fhsvk8rqt6880000gn/T/833833d6f66e47b68d936abe5d707758-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_m/zyrbphls3w34fhsvk8rqt6880000gn/T/833833d6f66e47b68d936abe5d707758-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 70 RHS
At line 112 BOUNDS
At line 115 ENDATA
Problem MODEL has 41 rows, 2 columns and 17 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 540 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node cha

In [4]:
import pulp
import random

ALL_SURGERIES_DATA = {
    101: {"duration": 120, "surgeon": "Dr_A", "deadline": 3},
    102: {"duration": 120, "surgeon": "Dr_B", "deadline": 3},
    103: {"duration": 300, "surgeon": "Dr_A", "deadline": 1},
    104: {"duration": 60,  "surgeon": "Dr_A", "deadline": 5},
    105: {"duration": 180, "surgeon": "Dr_B", "deadline": 5},
}


def get_initial_schedules():
    """
    Creates one or more simple schedules to start the algorithm.
    This is a "dummy" heuristic.
    """
    schedule_1 = {
        "id": "Initial_Sched_1_Mon",
        "B_j": 120,
        "day": "Mon",
        "surgeries": [101], # Contains one mandatory surgery
        "surgeon_work": {"Dr_A": 120},
        "surgeon_busy_times": {
            ("Dr_A", "Mon", 9): 1, ("Dr_A", "Mon", 10): 1
        }
    }
    schedule_2 = {
        "id": "Initial_Sched_2_Mon",
        "B_j": 300,
        "day": "Mon",
        "surgeries": [103], # Contains another mandatory surgery
        "surgeon_work": {"Dr_A": 300},
        "surgeon_busy_times": {
            ("Dr_A", "Mon", 11): 1, ("Dr_A", "Mon", 12): 1, ("Dr_A", "Mon", 13): 1,
            ("Dr_A", "Mon", 14): 1, ("Dr_A", "Mon", 15): 1
        }
    }
    schedule_3 = {
        "id": "Initial_Sched_3_Tue",
        "B_j": 120,
        "day": "Tue",
        "surgeries": [102], # Contains the third mandatory surgery
        "surgeon_work": {"Dr_B": 120},
        "surgeon_busy_times": {
            ("Dr_B", "Tue", 9): 1, ("Dr_B", "Tue", 10): 1
        }
    }
    return [schedule_1, schedule_2, schedule_3]

def extract_dual_prices(prob):
    """
    Extracts all dual prices (pi) from the solved master LP.
    """
    dual_prices = {}
    for name, c in prob.constraints.items():
        dual_prices[name] = c.pi
    return dual_prices

def solve_subproblem_stub(day, dual_prices, all_surgeries_data):
    """
    This is a FAKE subproblem solver.
    It "pretends" to build and solve the complex CP model.
    It randomly decides if it "found" a new, profitable schedule.
    """
    print(f"  ...Solving Subproblem for {day}...")
    
    # In a REAL problem, you would:
    # 1. Build a CP model (using ortools.sat).
    # 2. Use dual_prices to build the reduced cost objective (Eq. 18).
    # 3. Solve the CP model to find the schedule with the
    #    MAXIMUM positive reduced cost.
    
    # FAKE logic: 50% chance of finding a "new schedule"
    if random.random() > 0.5:
        
        # "Found" a new profitable schedule.
        # Let's create a new "fake" schedule.
        surgery_id = random.choice(list(all_surgeries_data.keys()))
        surg_data = all_surgeries_data[surgery_id]
        
        new_schedule = {
            "id": f"Generated_Sched_{random.randint(100,999)}_{day}",
            "B_j": surg_data["duration"],
            "day": day,
            "surgeries": [surgery_id],
            "surgeon_work": {surg_data["surgeon"]: surg_data["duration"]},
            "surgeon_busy_times": {
                # This is a simplified "busy" calculation
                (surg_data["surgeon"], day, 9): 1 
            }
        }
        
        # We "pretend" its reduced cost is positive
        reduced_cost = 1.0 
        
        print(f"  ...FOUND new schedule {new_schedule['id']} with RC > 0.")
        return new_schedule, reduced_cost
        
    else:
        # Did not find any schedule with a positive reduced cost
        print(f"  ...No new profitable schedules found for {day}.")
        return None, 0.0

# --- 4. MAIN COLUMN GENERATION LOOP ---
if __name__ == "__main__":
    
    # --- Initialization ---
    known_schedules = get_initial_schedules()
    iteration = 0
    
    # --- The "Price" (Column Generation) Loop ---
    # This loop is the "Price" part of Branch-and-Price.
    # We will loop as long as we keep finding new, profitable schedules.
    while True:
        iteration += 1
        print(f"\n--- COLUMN GENERATION: ITERATION {iteration} ---")
        
        # --- 1. Solve the Master Problem (LP) ---
        print("Solving Master Problem (LP) with {len(known_schedules)} schedules...")
        master_lp = build_master_lp(known_schedules, MANDATORY_SURGERIES, OPTIONAL_SURGERIES,
                                    ALL_SURGEONS, ALL_DAYS, ALL_TIMES, K_d, A_ld)
        master_lp.solve()
        
        if master_lp.status != pulp.LpStatusOptimal:
            print("Master LP failed to solve. Stopping.")
            break
            
        # --- 2. Get Dual Prices (pi) ---
        # These prices are the *output* of the Master Problem
        # and the *input* to the Subproblem.
        dual_prices = extract_dual_prices(master_lp)
        
        # --- 3. Solve Subproblems (one for each day) ---
        new_schedules_found = False
        for day in ALL_DAYS:
            # Call the subproblem (our fake stub)
            new_schedule, reduced_cost = solve_subproblem_stub(day, dual_prices, ALL_SURGERIES_DATA)
            
            # If the reduced cost is positive, we found a good schedule!
            if reduced_cost > 1e-6:
                new_schedules_found = True
                # Add the new schedule to our master list
                if new_schedule not in known_schedules:
                     known_schedules.append(new_schedule)
                     
        # --- 4. Check for Convergence ---
        if not new_schedules_found:
            print("\n--- CONVERGENCE REACHED ---")
            print("Subproblems found no new profitable schedules.")
            print("Column Generation loop is finished.")
            break # Exit the while loop
        
        # Safety break for this example
        if iteration > 5:
            print("\nReached max iterations for example. Stopping.")
            break

    # --- 5. FINAL INTEGER SOLVE ---
    # The loop is done. 'known_schedules' now has all the "good" schedules.
    # Now we solve ONE LAST TIME, but as an Integer Problem.
    
    print(f"\nBuilding FINAL problem with {len(known_schedules)} total schedules...")
    final_prob = build_master_lp(known_schedules, MANDATORY_SURGERIES, OPTIONAL_SURGERIES,
                                 ALL_SURGEONS, ALL_DAYS, ALL_TIMES, K_d, A_ld)
    
    # --- Change variables from Continuous to Integer ---
    for v in final_prob.variables():
        v.cat = 'Integer'
        v.upBound = 1 # A schedule can be used at most once
        v.lowBound = 0

    print("Solving the FINAL Integer Problem for the real answer...")
    final_prob.solve()

    # --- 6. PRINT THE REAL ANSWER ---
    print(f"\n--- FINAL SOLUTION ---")
    print(f"Status: {pulp.LpStatus[final_prob.status]}")
    
    if final_prob.status == pulp.LpStatusOptimal:
        print("\n--- FINAL OR ROOM ALLOCATION ---")
        total_time = 0
        for v in final_prob.variables():
            if v.value() > 0.9: # i.e., if the variable = 1.0
                print(f"  USE SCHEDULE: {v.name}")
                # Find the schedule object to print details
                for s in known_schedules:
                    if s["id"] == v.name.replace("Schedule_", ""):
                        print(f"    - Day: {s['day']}, Surgeries: {s['surgeries']}, Time: {s['B_j']} min")
                        total_time += s['B_j']
        print(f"\nTotal Scheduled Time: {total_time} minutes")
    else:
        print("\nNo valid integer schedule allocation found.")


--- COLUMN GENERATION: ITERATION 1 ---
Solving Master Problem (LP) with {len(known_schedules)} schedules...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/matthewkeller/4A/bme-411/411-final-project/or-env/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/_m/zyrbphls3w34fhsvk8rqt6880000gn/T/e5199630ee614322a452cba8acb9bc5a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_m/zyrbphls3w34fhsvk8rqt6880000gn/T/e5199630ee614322a452cba8acb9bc5a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 68 RHS
At line 110 BOUNDS
At line 111 ENDATA
Problem MODEL has 41 rows, 3 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-41) rows, 0 (-3) columns and 0 (-18) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 540
After Postsolve, obj

In [6]:
!pip install ortools

  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 18.3 MB/s eta 0:00:0031m19.3 MB/s eta 0:00:01
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [ortools]━━━ 7/8 [ortools]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [7]:
from ortools.sat.python import cp_model

# --- THIS IS THE REAL SUBPROBLEM IMPLEMENTATION ---
def solve_subproblem(
    day, 
    dual_prices, 
    all_surgeries_data, 
    all_surgeons,
    all_times,
    A_ld
):
    """
    Solves the subproblem for a single 'day' using a CP-SAT model.
    This finds the new schedule (column) with the highest positive reduced cost.
    """
    
    # --- 1. PREPROCESSING (Calculate Lookup Tables) ---
    # This is the logic from Eq. (17) in the paper. We are converting
    # the dual prices into simple "profit" and "cost" tables.
    
    # G_di[i] = "Base profit of *doing* surgery i"
    G_di = {0: 0} # 0 is the "null" surgery, it has 0 profit/cost
    for i, surg in all_surgeries_data.items():
        # Get the dual prices related to this surgery
        pi_i = (
            dual_prices.get(f"Pi_i_Mandatory_{i}", 0) + 
            dual_prices.get(f"Pi_i_Optional_{i}", 0)
        )
        pi_ld = dual_prices.get(f"Pi_ld_Surgeon_Hours_{surg['surgeon']}_{day}", 0)
        t_i = surg['duration'] # Duration of the surgery
        
        # G_di = t_i - pi_i - (t_i * pi_ld)
        G_di[i] = t_i - pi_i - (t_i * pi_ld)

    # pi_dit_star[(i, t)] = "Start-time cost of *starting* surgery i at time t"
    pi_dit_star = {} 
    for i, surg in all_surgeries_data.items():
        for t_start in all_times:
            cost = 0.0
            # A time slot in the paper is a discrete unit (e.g., 5 min)
            # Here, let's assume ALL_TIMES is in minutes and duration is in minutes.
            # This is a simplified calculation.
            t_end = t_start + surg['duration']
            
            for t_busy in range(t_start, t_end):
                if t_busy in all_times: # Only count costs within the valid time range
                    cost += dual_prices.get(f"Pi_ldt_Surgeon_Overlap_{surg['surgeon']}_{day}_{t_busy}", 0)
            
            pi_dit_star[(i, t_start)] = cost

    # Get the fixed cost for using an OR on this day
    pi_d_cost = dual_prices.get(f"Pi_d_OR_Limit_{day}", 0)

    # --- 2. MODEL & VARIABLE SETUP ---
    model = cp_model.CpModel()

    # 'n' = max number of surgeries we can pack into one OR day.
    # We must set an upper bound. Let's say 8.
    num_positions = 8 
    
    # Get all possible surgery IDs, including 0 for "null"
    surgery_ids = [0] + list(all_surgeries_data.keys())
    day_duration = all_times[-1] + 1 # e.g., 480 minutes
    
    # W_p: The ID of the surgery assigned to position p (Eq. 8)
    W = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(surgery_ids), f'W_{p}') 
         for p in range(num_positions)]
    
    # V_p: The start time (in minutes) of the surgery at position p (Eq. 9)
    V = [model.NewIntVar(0, day_duration, f'V_{p}') 
         for p in range(num_positions)]

    # --- 3. ADD CONSTRAINTS (Eq. 10-15) ---
    # This is the most complex part of the CP model.

    # Eq (13): Each surgery used at most once.
    # For each surgery ID, count how many positions (W_p) are assigned to it.
    # The sum must be <= 1.
    for i in all_surgeries_data.keys():
        model.Add(sum(W[p] == i for p in range(num_positions)) <= 1)

    # Eq (10): If a position is empty, all later positions are empty
    # This "breaks symmetry" and makes the solver faster.
    for p in range(num_positions - 1):
        model.Add(W[p] == 0).OnlyEnforceIf(W[p+1] != 0) # A simpler form
        # Paper's form: (W_p == 0) => (W_p+1 == 0)
        # model.Add(W[p+1] == 0).OnlyEnforceIf(W[p] == 0)

    # Eq (11) & (14): Scheduling logic and surgeon limits.
    # This requires "Element" constraints to look up a surgery's duration
    # and surgeon based on the W_p variable.
    #
    # TODO: Implement Eq (11) (No Overlap)
    # This is the core scheduling logic. You need to:
    # 1. Create interval variables for each position.
    # 2. The duration of interval 'p' depends on W[p] (needs a lookup).
    # 3. Add non-overlapping constraints between intervals.
    # 4. Enforce V[p+1] >= V[p] + duration[W_p] + cleaning_time[W_p, W_p+1]
    # This is very advanced.
    
    # TODO: Implement Eq (14) (Max Surgeon Hours)
    # This is also very advanced. You need to:
    # 1. For each surgeon 'l', create a sum_work_l variable.
    # 2. Loop p=0..n: if surgeon_of[W_p] == l, add duration[W_p] to sum_work_l
    # 3. Add constraint: sum_work_l <= A_ld[(l, day)]

    # --- 4. DEFINE OBJECTIVE FUNCTION (Eq. 18) ---
    # Maximize Reduced Cost = sum(G_p) - sum(Pi_cost_p) - pi_d_cost
    
    # This also requires "Element" constraints to look up the
    # profit/cost based on the chosen W_p and V_p variables.
    
    # TODO: Create objective terms
    # 1. Create 'n' variables for the "profit" of each position: G_p
    # 2. Use model.AddElement(W[p], G_di_list, G_p) to look up the
    #    profit for the surgery chosen at position p.
    # 3. Create 'n' variables for the "start cost" of each position: Pi_cost_p
    # 4. This is a 2D lookup: Pi_cost_p = pi_dit_star[W[p], V[p]]. This
    #    requires advanced "Element" constraint usage.
    
    # For now, let's set a placeholder objective to make the code runnable
    # We'll just maximize the 'profit' part and ignore the hard constraints
    G_di_list = [G_di.get(i, 0) for i in range(max(surgery_ids) + 1)]
    G_p_vars = [model.NewIntVar(-10000, 10000, f'G_p_{p}') for p in range(num_positions)]
    for p in range(num_positions):
        # This is how you do a 1D lookup:
        model.AddElement(W[p], G_di_list, G_p_vars[p])
        
    # Placeholder objective:
    model.Maximize(sum(G_p_vars) - pi_d_cost)
    
    # The REAL objective would also subtract the pi_dit_star costs,
    # which is a much harder 2D lookup.

    # --- 5. SOLVE & PARSE ---
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        reduced_cost = solver.ObjectiveValue()
        
        # Only return a schedule if it's "profitable"
        if reduced_cost > 1e-6: # 1e-6 is a small tolerance for > 0
            
            # Re-create the new schedule from the solver's values
            new_schedule = {
                "id": f"Generated_Sched_{random.randint(100,999)}_{day}",
                "day": day,
                "surgeries": [],
                "surgeon_work": {},
                "surgeon_busy_times": {},
                "B_j": 0
            }
            
            for p in range(num_positions):
                surgery_id = solver.Value(W[p])
                if surgery_id > 0: # If it's not a "null" surgery
                    surg_data = all_surgeries_data[surgery_id]
                    start_time = solver.Value(V[p])
                    
                    new_schedule["surgeries"].append(surgery_id)
                    new_schedule["B_j"] += surg_data["duration"]
                    
                    # Add surgeon work
                    surg = surg_data["surgeon"]
                    new_schedule["surgeon_work"][surg] = \
                        new_schedule["surgeon_work"].get(surg, 0) + surg_data["duration"]
                    
                    # Add busy times
                    for t_busy in range(start_time, start_time + surg_data["duration"]):
                         if t_busy in all_times:
                            new_schedule["surgeon_busy_times"][(surg, day, t_busy)] = 1
            
            return new_schedule, reduced_cost

    # No solution found or no profitable solution found
    return None, 0.0